In [1]:
import os

folder_path = 'Dataset1.0/images'

# 获取文件夹中的所有文件名
file_names = os.listdir(folder_path)

# 将文件名写入到 train.txt 文件中
# with open(f'Dataset1.0/{whichset}.txt', 'w') as f:
with open(f'Dataset1.0/train.txt', 'w') as f:
    for file_name in file_names:
        f.write(os.path.splitext(file_name)[0] + '\n')


In [2]:
import os
import random
import xml.etree.ElementTree as ET
import numpy as np
from utils.utils import get_classes

classes_path        = 'model_data/vehicle_classes.txt'
classes, _          = get_classes(classes_path)
print("classes is ", classes)
whichpath           = "Dataset1.0"
nums                = np.zeros(len(classes))

classes is  ['car', 'truck']


In [3]:
import os
import random

def convert_yolo_to_custom_format(label_dir, image_dir, output_file, image_extension=".jpg", image_width=1280, image_height=720):
    """
    Converts YOLO format labels to custom format for train.txt.
    
    Args:
        label_dir (str): Path to the directory containing label files (.txt).
        image_dir (str): Path to the directory containing image files.
        output_file (str): Path to save the combined file.
        image_extension (str): Extension of image files, e.g., '.jpg'.
        image_width (int): Width of the image in pixels.
        image_height (int): Height of the image in pixels.
    """
    with open(output_file, 'w') as outfile:
        for label_file in sorted(os.listdir(label_dir)):
            if label_file.endswith(".txt"):
                # Determine the corresponding image file
                base_name = os.path.splitext(label_file)[0]
                image_path = os.path.join(image_dir, f"{base_name}{image_extension}")
                
                # Ensure the image file exists
                if not os.path.exists(image_path):
                    print(f"Warning: Image file {image_path} does not exist. Skipping.")
                    continue
                
                # Start writing the line with the image path
                outfile.write(image_path)
                
                # Read the label file
                label_path = os.path.join(label_dir, label_file)
                with open(label_path, 'r') as infile:
                    for line in infile:
                        class_id, center_x, center_y, width, height = map(float, line.split())
                        
                        # Convert YOLO normalized coordinates to pixel coordinates
                        x_min = int((center_x - width / 2) * image_width)
                        y_min = int((center_y - height / 2) * image_height)
                        x_max = int((center_x + width / 2) * image_width)
                        y_max = int((center_y + height / 2) * image_height)
                        
                        # Append the bounding box and class info
                        outfile.write(f" {x_min},{y_min},{x_max},{y_max},{int(class_id)}")
                
                # End the line
                outfile.write("\n")
    print(f"Conversion complete. Output saved to {output_file}")

def split_dataset(input_file, train_file, val_file, test_file, train_ratio=0.6, val_ratio=0.1, test_ratio=0.3):
    """
    Splits a dataset file into train, val, and test files based on given ratios.
    
    Args:
        input_file (str): Path to the combined dataset file.
        train_file (str): Path to save the training dataset.
        val_file (str): Path to save the validation dataset.
        test_file (str): Path to save the test dataset.
        train_ratio (float): Ratio of training data.
        val_ratio (float): Ratio of validation data.
        test_ratio (float): Ratio of test data.
    """
    with open(input_file, 'r') as infile:
        lines = infile.readlines()
    
    # Shuffle the data for random splitting
    random.shuffle(lines)
    
    # Calculate split indices
    total = len(lines)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    
    # Split the dataset
    train_data = lines[:train_end]
    val_data = lines[train_end:val_end]
    test_data = lines[val_end:]
    
    # Write to respective files
    with open(train_file, 'w') as train_out:
        train_out.writelines(train_data)
    
    with open(val_file, 'w') as val_out:
        val_out.writelines(val_data)
    
    with open(test_file, 'w') as test_out:
        test_out.writelines(test_data)
    
    print(f"Dataset split complete:")
    print(f"  Training data: {len(train_data)} samples")
    print(f"  Validation data: {len(val_data)} samples")
    print(f"  Test data: {len(test_data)} samples")

# Example usage
label_dir = "Dataset1.0/labels"
image_dir = "Dataset1.0/images"
output_file = "Dataset1.0/train.txt"
train_file = "Dataset1.0/train_split.txt"
val_file = "Dataset1.0/val_split.txt"
test_file = "Dataset1.0/test_split.txt"
image_width = 1920  # Replace with actual image width
image_height = 1080  # Replace with actual image height

# Convert YOLO labels to custom format
convert_yolo_to_custom_format(label_dir, image_dir, output_file, image_extension=".jpg", image_width=image_width, image_height=image_height)

# Split the dataset into train, val, and test sets
split_dataset(output_file, train_file, val_file, test_file, train_ratio=0.6, val_ratio=0.1, test_ratio=0.3)


Conversion complete. Output saved to Dataset1.0/train.txt
Dataset split complete:
  Training data: 8556 samples
  Validation data: 1426 samples
  Test data: 4278 samples


### Below is not used by 11/25.

In [18]:
def convert_annotation(image_id, list_file, image_set):
    

    in_file = open(os.path.join(f'{whichpath}/%slabel/%s.xml'%(image_set, image_id)), encoding='utf-8')


    tree=ET.parse(in_file)
    root = tree.getroot()


    for obj in root.iter('object'):

        difficult = 0 
        if obj.find('difficult')!=None:
            difficult = obj.find('difficult').text

        cls = obj.find('name').text
        if  int(difficult)==1 or cls not in classes:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)), int(float(xmlbox.find('ymin').text)), int(float(xmlbox.find('xmax').text)), int(float(xmlbox.find('ymax').text)))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))
        
        nums[classes.index(cls)] = nums[classes.index(cls)] + 1

In [19]:

random.seed(0)
Sets = [('train'),('val'),('test')]


type_index = 0
for image_set in Sets:
    image_ids = open(os.path.join(f'{whichpath}/%s.txt'%(image_set)), encoding='utf-8').read().strip().split()
    list_file = open(f'{whichpath}/%slabel.txt'%(image_set), 'w', encoding='utf-8')
    for image_id in image_ids:
        list_file.write(f'{whichpath}/%s/%s.jpg'%(image_set, image_id))

        convert_annotation(image_id, list_file, image_set)
        list_file.write('\n')
    type_index += 1
    list_file.close()


def printTable(List1, List2):
    for i in range(len(List1[0])):
        print("|", end=' ')
        for j in range(len(List1)):
            print(List1[j][i].rjust(int(List2[j])), end=' ')
            print("|", end=' ')
        print()

str_nums = [str(int(x)) for x in nums]
tableData = [
    classes, str_nums
]
colWidths = [0]*len(tableData)
len1 = 0
for i in range(len(tableData)):
    for j in range(len(tableData[i])):
        if len(tableData[i][j]) > colWidths[i]:
            colWidths[i] = len(tableData[i][j])
printTable(tableData, colWidths)

| bsu | 344 | 
| ecl | 954 | 
|  lm | 759 | 
